In [1]:
import os
import pickle

import pandas as pd

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout

from src.data_preparation.data_preprocessing import DataReshaperLSTM
from src.visualization.charts import ModelCharts
from src.visualization.metrics import PredictionEvaluator
from src.config import Config

## Load the data

In [2]:
config = Config()
variant_co2 = 'co2_per_capita'
train_df = pd.read_csv(os.path.join(config.output_cleaned_lstm, f'{variant_co2}/train.csv')).set_index(["year", config.additional_index]).drop(columns=['country_order'])
test_df = pd.read_csv(os.path.join(config.output_cleaned_lstm, f'{variant_co2}/test.csv')).set_index(["year", config.additional_index]).drop(columns=['country_order'])

with open(os.path.join(config.output_cleaned_lstm, f'{variant_co2}/data_preprocessor_lstm.pkl'), 'rb') as f:
    data_preprocessor_lstm = pickle.load(f)

In [3]:
train_df

,,country_t-2,population_t-2,gdp_t-2,temperature_change_from_co2_t-2,total_ghg_t-2,cement_co2_t-2,coal_co2_t-2,flaring_co2_t-2,gas_co2_t-2,land_use_change_co2_t-2,...,gdp_t-1,temperature_change_from_co2_t-1,total_ghg_t-1,cement_co2_t-1,coal_co2_t-1,flaring_co2_t-1,gas_co2_t-1,land_use_change_co2_t-1,oil_co2_t-1,co2
year,country_index,,,,,,,,,,,,,,,,,,,,,
1853,Stany Zjednoczone,34,0.016985,0.078879,0.000000,0.809861,0.000000,0.065960,0.000000,0.000000,0.489399,...,0.083750,0.054411,0.826034,0.000000,0.069644,0.000000,0.000000,0.501986,0.000000,0.495427
1854,Stany Zjednoczone,34,0.017544,0.083750,0.054411,0.826034,0.000000,0.069644,0.000000,0.000000,0.501986,...,0.090790,0.052824,0.807592,0.000000,0.076120,0.000000,0.000000,0.494294,0.000000,0.500984
1855,Stany Zjednoczone,34,0.018120,0.090790,0.052824,0.807592,0.000000,0.076120,0.000000,0.000000,0.494294,...,0.090836,0.102565,0.813081,0.000000,0.081242,0.000000,0.000000,0.498800,0.000000,0.495014
1856,Stany Zjednoczone,34,0.018712,0.090836,0.102565,0.813081,0.000000,0.081242,0.000000,0.000000,0.498800,...,0.088710,0.099573,0.799701,0.000000,0.090770,0.000000,0.000000,0.491227,0.000000,0.495772
1857,Stany Zjednoczone,34,0.019322,0.088710,0.099573,0.799701,0.000000,0.090770,0.000000,0.000000,0.491227,...,0.090730,0.096669,0.799440,0.000000,0.092454,0.000000,0.000000,0.491663,0.000000,0.488626
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1996,Szwecja,35,0.005112,0.633681,0.310355,0.169580,0.160121,0.083308,0.001888,0.034710,0.056927,...,0.657400,0.308734,0.176222,0.176169,0.082777,0.002145,0.035107,0.062237,0.286368,0.123911
1997,Szwecja,35,0.005147,0.657400,0.308734,0.176222,0.186143,0.082777,0.002145,0.035107,0.062237,...,0.668932,0.308244,0.184783,0.166254,0.091217,0.002047,0.037824,0.059806,0.302837,0.118362
1998,Szwecja,35,0.005158,0.668932,0.308244,0.184783,0.175666,0.091217,0.002047,0.037824,0.059806,...,0.688825,0.308063,0.176055,0.149879,0.079360,0.002274,0.038312,0.061110,0.280898,0.119892


## Reshape the data

In [4]:
data_resherper = DataReshaperLSTM()
x_train, x_test, y_train, y_test = data_resherper.reshape_data(train_df, test_df)

## Build the model

In [5]:
input_shape = (x_train.shape[1], x_train.shape[2])
output_units = 1

model = Sequential([
    LSTM(50, input_shape=input_shape, return_sequences = True),
    Dropout(0.2),
    LSTM(50),
    Dropout(0.2),
    Dense(output_units, activation='relu')
])

model.compile(optimizer="adam", loss="mean_squared_error")
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 1, 50)             14600     
                                                                 
 dropout (Dropout)           (None, 1, 50)             0         
                                                                 
 lstm_1 (LSTM)               (None, 50)                20200     
                                                                 
 dropout_1 (Dropout)         (None, 50)                0         
                                                                 
 dense (Dense)               (None, 1)                 51        
                                                                 
Total params: 34,851
Trainable params: 34,851
Non-trainable params: 0
_________________________________________________________________


## Train the model

In [6]:
history = model.fit(x_train, y_train, epochs=config.epochs, batch_size=config.batch_size)

Epoch 1/15
3153/3153 [==============================] - 4s 810us/step - loss: 0.0048
Epoch 2/15
3153/3153 [==============================] - 2s 781us/step - loss: 0.0025
Epoch 3/15
3153/3153 [==============================] - 3s 921us/step - loss: 0.0020
Epoch 4/15
3153/3153 [==============================] - 3s 897us/step - loss: 0.0021
Epoch 5/15
3153/3153 [==============================] - 2s 738us/step - loss: 0.0018
Epoch 6/15
3153/3153 [==============================] - 2s 735us/step - loss: 0.0017
Epoch 7/15
3153/3153 [==============================] - 2s 726us/step - loss: 0.0015
Epoch 8/15
3153/3153 [==============================] - 2s 693us/step - loss: 0.0017
Epoch 9/15
3153/3153 [==============================] - 2s 673us/step - loss: 0.0014
Epoch 10/15
3153/3153 [==============================] - 2s 680us/step - loss: 0.0015
Epoch 11/15
3153/3153 [==============================] - 2s 704us/step - loss: 0.0014
Epoch 12/15
3153/3153 [==============================] - 2s 709

In [7]:
# Save the model
model.save(os.path.join(config.models_folder, f'{variant_co2}_lstm_model.h5'))

# Evaluate the model on the test set
loss = model.evaluate(x_test, y_test, verbose = 0)

## Predictions

In [8]:
train_predictions = model.predict(x_train)
test_predictions = model.predict(x_test)

inverted_data_predicted_train_y = data_preprocessor_lstm.inverse_transform_data(train_predictions, train_predictions.shape[0], train_df.shape[1]-2)
inverted_data_train_y = data_preprocessor_lstm.inverse_transform_data(y_train, train_predictions.shape[0], train_df.shape[1]-2)

inverted_data_predicted_test_y = data_preprocessor_lstm.inverse_transform_data(test_predictions, test_predictions.shape[0], test_df.shape[1]-2)
inverted_data_test_y = data_preprocessor_lstm.inverse_transform_data(y_test, test_predictions.shape[0], test_df.shape[1]-2)

26/26 [==============================] - 0s 450us/step


In [9]:
train_df_reset = train_df.reset_index()
test_df_reset = test_df.reset_index()

co2_predicted_train = inverted_data_predicted_train_y[:, -1]
co2_actual_train = inverted_data_train_y[:, -1]
co2_predicted_test = inverted_data_predicted_test_y[:, -1]
co2_actual_test = inverted_data_test_y[:, -1]

# Create DataFrames for train and test
train_results = pd.DataFrame({
    "country": train_df_reset["country_index"].values,
    "year": train_df_reset["year"].values,
    "co2_predicted": co2_predicted_train,
    "co2_actual": co2_actual_train
})

test_results = pd.DataFrame({
    "country": test_df_reset["country_index"].values,
    "year": test_df_reset["year"].values,
    "co2_predicted": co2_predicted_test,
    "co2_actual": co2_actual_test
})


train_results.to_csv(os.path.join(config.predictions_lstm, f'{variant_co2}_train.csv'))
test_results.to_csv(os.path.join(config.predictions_lstm, f'{variant_co2}_test.csv'))

## Charts

In [10]:
charts = ModelCharts(train_results, test_results)

#Line plot
charts.generate_line_plot(variant = variant_co2, model_output_file=config.predictions_lstm)
charts.generate_line_plot_one_dataset(variant = variant_co2, model_output_file=config.predictions_lstm, dataset_type='train')
charts.generate_line_plot_one_dataset(variant = variant_co2, model_output_file=config.predictions_lstm, dataset_type='test')

#Scatter plot
charts.generate_scatter_plot(variant = variant_co2, model_output_file=config.predictions_lstm)
charts.generate_scatter_plot_one_dataset(variant = variant_co2, model_output_file=config.predictions_lstm, dataset_type='train')
charts.generate_scatter_plot_one_dataset(variant = variant_co2, model_output_file=config.predictions_lstm, dataset_type='test')

## Metrics

In [11]:
evaluator = PredictionEvaluator()
evaluator.evaluate(train_results, test_results, actual_col='co2_actual', predicted_col='co2_predicted', variant = variant_co2, model_output_file=config.metrics_lstm)